In [1]:
import torch
from torch import nn
from torchvision import transforms,datasets
torch.manual_seed(1)

In [2]:
import numpy as np
np.random.seed(1)

In [3]:
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

### Data set loading

In [4]:
train_data = datasets.CIFAR10(root = './data',transform=transforms.ToTensor(),download=True)
test_data  = datasets.CIFAR10(root = './data',train=False,transform=transforms.ToTensor(),download=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
test_data[0][0].shape

torch.Size([3, 32, 32])

### Defining batches

In [6]:
BATCH_SIZE = 500
EPOCHS = 800

train_loader = torch.utils.data.DataLoader(train_data,BATCH_SIZE,True)
test_loader = torch.utils.data.DataLoader(test_data,BATCH_SIZE)

### CNN model class

In [7]:
class CNN(nn.Module):
    def __init__(self,inp_channels=3,input_size=32,output_size=10):
        super(CNN, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=inp_channels,out_channels=64,kernel_size=3)
        self.relu1 = nn.ReLU()
        
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3)
        self.relu2 = nn.ReLU()
                
        # Maxpool1
#         self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        # Convolution 3
        self.cnn3 = nn.Conv2d(in_channels=64,out_channels=32,kernel_size=3)
        self.relu3 = nn.ReLU()
        
        # Maxpool 1
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)
                
        # Convolution 4
        self.cnn4 = nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3)
        self.relu4 = nn.ReLU()
        self.cnn4_drop = nn.Dropout2d(0.2)
        
        # Convolution 5
        self.cnn5 = nn.Conv2d(in_channels=32,out_channels=16,kernel_size=3)
        self.relu5 = nn.ReLU()
        
        self.cnn6 = nn.Conv2d(in_channels=16,out_channels=16,kernel_size=3)
        self.relu6 = nn.ReLU()
        
        # Fully Connected 1
        self.fc1 = nn.Linear(16*7*7,output_size)
#         self.relu6 = nn.ReLU()
#         self.fc2 = nn.Linear(128,output_size)
        
        

        
    def forward(self,x):
        # x = 100,3,32,32
        
        # Convolution 1
        out = self.cnn1(x) # out = 100,64,30,30
        out = self.relu1(out)
        
        # Convolution 2
        out = self.cnn2(out) # out = 100,64,28,28
        out = self.relu2(out)
        
        out = self.cnn3(out) # out = 100,32,26,26
        out = self.relu3(out)
        out = self.maxpool3(out) # out = 100,32,13,13
        
        out = self.cnn4(out) # out = 100,32,11,11
        out = self.relu4(out)
        
        out = self.cnn5(out) # out = 100,16,9,9
        out = self.relu5(out)
        
        out = self.cnn6(out) # out = 100,16,7,7
        out = self.relu6(out)
        
        out = out.view(out.size(0),-1)
        
        # Fully Connected layer 1
        out = self.fc1(out)
#         out = self.relu6(out)
#         out = self.fc2(out)
        
        return out

model = CNN()

if torch.cuda.is_available():
    model.cuda()
learning_rate = 0.05
 
optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate,rho=0.95,weight_decay=0.005)

criterion = nn.CrossEntropyLoss()

loss_save = torch.zeros(int(EPOCHS*len(train_data)/BATCH_SIZE))

In [8]:
def accuracy(loader,model):
    # Calculate Accuracy         
    correct = 0
    total = 0
    # Iterate through test dataset
    for images, labels in loader:
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            images = images.cuda()

        # Forward pass only to get logits/output
        outputs = model(images)

        # Get predictions from the maximum value
        _, predicted = torch.max(outputs.data, 1)

        # Total number of labels
        total += labels.size(0)

        # Total correct predictions
        correct += (predicted.cpu() == labels.cpu()).sum().float()


    accuracy = 100. * correct / total
    return accuracy

%%time
iter = 0
for epoch in range(EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
         
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()

        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        outputs = model(images)
         
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        loss_save[iter] = loss.item()
        
        # Getting gradients w.r.t. parameters
        loss.backward()
         
        # Updating parameters
        optimizer.step()
         
        iter += 1
         
        if iter % 1000 == 0:
            # Print Loss
            test_accuracy = accuracy(test_loader,model)
            train_accuracy = accuracy(train_loader,model)
            print('Iteration: {}. Loss: {}, Test Accuracy: {},Train Accuracy: {}'.format(iter, loss.item(), test_accuracy,train_accuracy))

In [ ]:
%%capture cap --no-stderr
TOTAL_PARAMS = 20
param_loss = np.zeros((TOTAL_PARAMS))
param_test_accuracy = np.zeros((TOTAL_PARAMS))
param_train_accyracy = np.zeros((TOTAL_PARAMS))
# selecting learning rate between 0.005 and 0.5
r = -2*np.random.random_sample((TOTAL_PARAMS)) - 1
learning_rate = np.power(10,r)*5

# selecting momentum between 1 - 0.001 and 1 - 0.1
r = -2*np.random.random_sample((TOTAL_PARAMS)) - 1
momentum = 1 - np.power(10,r)

# selecting weight decay between 0.001 and 0.01
r = np.random.random_sample((TOTAL_PARAMS)) - 3
decay = np.power(10,r)

for param_iter in range(TOTAL_PARAMS):
    model = CNN()

    if torch.cuda.is_available():
        model.cuda()
    
    optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate[param_iter],rho=momentum[param_iter],weight_decay=decay[param_iter])

    criterion = nn.CrossEntropyLoss()
    loss_save = torch.zeros(int(EPOCHS*len(train_data)/BATCH_SIZE))
    
    iter = 0
    for epoch in range(EPOCHS):
        for i, (images, labels) in enumerate(train_loader):

            #######################
            #  USE GPU FOR MODEL  #
            #######################
            if torch.cuda.is_available():
                images = images.cuda()
                labels = labels.cuda()


            # Clear gradients w.r.t. parameters
            optimizer.zero_grad()

            # Forward pass to get output/logits
            outputs = model(images)

            # Calculate Loss: softmax --> cross entropy loss
            loss = criterion(outputs, labels)
            loss_save[iter] = loss.item()

            # Getting gradients w.r.t. parameters
            loss.backward()

            # Updating parameters
            optimizer.step()

            iter +=1
            if iter % 1000 == 0:
                # Print Loss
                test_accuracy = accuracy(test_loader,model)
                train_accuracy = accuracy(train_loader,model)
                print('Iteration: {}. Loss: {}, Test Accuracy: {},Train Accuracy: {}'.format(iter, loss.item(), test_accuracy,train_accuracy))
    
    
    test_accuracy = accuracy(test_loader,model)
    train_accuracy = accuracy(train_loader,model)
    print('Learning Rate:{}, Momentum:{}, Weight decay:{}'.format(learning_rate[param_iter],momentum[param_iter],decay[param_iter]))
    print('Loss: {}, Test Accuracy: {},Train Accuracy: {}'.format(loss.item(), test_accuracy,train_accuracy))
                            
    ax = sns.lineplot(data = loss_save.numpy())
    ax.set(xlabel = "Iterations", ylabel="Loss")
    plt.show()
    
with open('output.txt', 'w') as out:
   out.write(cap.stdout)

In [ ]:
cap.show()

In [ ]:

ax = sns.lineplot(data = loss_save.numpy())
ax.set(xlabel = "Iterations", ylabel="Loss")
plt.show()